In [1]:
import pandas as pd #the csv reading library
import matplotlib.pyplot as plt #the ploting library
import glob #to get file names
import numpy as np
from pandas import Series
import seaborn as sns
from matplotlib.pyplot import figure
import cmocean.cm as cmo
import seaborn as sns

from pylab import rcParams
from numpy import diff
from scipy import interpolate
import scipy.stats as stats

from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols

import matplotlib.gridspec as gridspec
from IPython.display import display, Math

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [2]:
df_iron = pd.read_csv('../Dissertation_projects/Data/IRON/Iron.csv', sep=',', index_col=0)

#make all the spots labeled NA a nan value
df_iron = df_iron.replace('NA', np.nan)
df_iron = df_iron.replace('NA ', np.nan)

#make all BDL a 0 
df_iron.loc[df_iron['FeT_uM'] == 'BLD', 'FeT_uM'] = 0
df_iron.loc[df_iron['Fe2_uM'] == 'BLD', 'Fe2_uM'] = 0

# convert columns to numeric data type
df_iron['FeT_uM'] = pd.to_numeric(df_iron['FeT_uM'], errors='coerce')
df_iron['Fe2_uM'] = pd.to_numeric(df_iron['Fe2_uM'], errors='coerce')

df_iron['FeT_uM'] = df_iron['FeT_uM'].where(pd.notna(df_iron['FeT_uM']), other=np.nan)
df_iron['Fe2_uM'] = df_iron['Fe2_uM'].where(pd.notna(df_iron['Fe2_uM']), other=np.nan)

#make all negative a 0 
df_iron.loc[df_iron['FeT_uM'] < 0, 'FeT_uM'] = 0
df_iron.loc[df_iron['Fe2_uM'] < 0, 'Fe2_uM'] = 0

#calculate Fe +3 
df_iron['Fe3_uM'] = df_iron['FeT_uM'] - df_iron['Fe2_uM']
df_iron.loc[df_iron['Fe3_uM'] < 0, 'Fe3_uM'] = 0
df_iron.loc[df_iron['Fe2_uM'] > df_iron['FeT_uM'], 'Fe3_uM'] = 0
df_iron['Fe3_uM'] = pd.to_numeric(df_iron['Fe3_uM'], errors='coerce')
df_iron['Fe3_uM'] = df_iron['Fe3_uM'].where(pd.notna(df_iron['Fe3_uM']), other=np.nan)

#time feild 
df_iron['Fe2_Time'] = pd.to_numeric(df_iron['Fe2_Time'], errors='coerce')
df_iron['Fe2_Time'] = df_iron['Fe2_Time'].where(pd.notna(df_iron['Fe2_Time']), other=np.nan)

#time feild 
df_iron['FeT_Time'] = pd.to_numeric(df_iron['FeT_Time'], errors='coerce')
df_iron['FeT_Time'] = df_iron['FeT_Time'].where(pd.notna(df_iron['FeT_Time']), other=np.nan)

# convert columns to numeric data type
df_iron['FeT_Depth'] = pd.to_numeric(df_iron['Fe2_Depth'], errors='coerce')
df_iron['Fe2_Depth'] = pd.to_numeric(df_iron['Fe2_Depth'], errors='coerce')

df_iron['code'] = df_iron['Location'] + '_' + df_iron['Campaign']

In [3]:
df_doc = pd.read_csv('../Dissertation_projects/Data/TOC/TOC_GOM.csv', sep=',', index_col=0)

#make all BDL a 0 
df_doc.loc[df_doc['NPOC_uM'] == 'BLD', 'NPOC_uM'] = 0
df_doc.loc[df_doc['TN_uM'] == 'BLD', 'TN_uM'] = 0

df_doc.loc[df_doc['NPOC_uM'] < 0, 'NPOC_uM'] = 0
df_doc.loc[df_doc['TN_uM'] < 0, 'TN_uM'] = 0

#fix date time 
df_doc['date_time'] = pd.to_datetime(df_doc['Date'])
df_doc['Date'] = df_doc['date_time'].dt.date

df_doc['NPOC_uM'] = pd.to_numeric(df_doc['NPOC_uM'], errors='coerce')
df_doc['NPOC_uM'] = df_doc['NPOC_uM'].where(pd.notna(df_doc['NPOC_uM']), other=np.nan)

df_doc['TN_uM'] = pd.to_numeric(df_doc['TN_uM'], errors='coerce')
df_doc['TN_uM'] = df_doc['TN_uM'].where(pd.notna(df_doc['TN_uM']), other=np.nan)

df_doc['Time'] = pd.to_numeric(df_doc['Time'], errors='coerce')
df_doc['Time'] = df_doc['Time'].where(pd.notna(df_doc['Time']), other=np.nan)

df_doc['code'] = df_doc['Location'] + '_' + df_doc['Campaign']

In [4]:
df_meta = pd.read_csv('../Dissertation_projects/Data/meta_data.csv', sep=',', index_col=0)

#fix date time 
df_meta['date_time'] = pd.to_datetime(df_meta['Date'])
df_meta['Date'] = df_meta['date_time'].dt.date

df_meta['code'] = df_meta['Station'] + '_' + df_meta['Trip']

In [5]:
df_c = pd.read_csv('../Dissertation_projects/Data/CDOM/Outputs.csv', sep=',', index_col=0)

df_c['Sample_idx'] = df_c.index.astype(object)
df_meta['Sample_idx'] = df_meta.index.astype(object)

df_c.index = df_c['file_id']

df_cdom = df_c.merge(df_meta, on="Sample_idx", how="left")

df_cdom.index = df_cdom['Sample_idx']
df_cdom['Sample'] = df_cdom.index
df_cdom.index = df_cdom['Sample']
df_cdom = df_cdom.drop('Sample_idx', axis = 1)
df_meta = df_meta.drop('Sample_idx', axis = 1)

In [6]:
df_por = pd.read_csv('../Dissertation_projects/Data/Porosity/Porosity.csv', sep=',', index_col = 0)

df_por['code'] = df_por['Stn'] + '_' + df_por['camp']

In [7]:
df_hights = pd.read_csv('../Dissertation_projects/Data/Hights.csv', sep=',')

df_hights['code'] = df_hights['Stn'] + '_' + df_hights['Camp']
#df_hights.drop("Stn", axis=1, inplace=True)
#df_hights.drop("Camp", axis=1, inplace=True)
df_hights.index = df_hights['code']
df_hights.drop("code", axis=1, inplace=True)

In [8]:
df_spectra_bc = pd.read_csv('../Dissertation_projects/Data/CDOM/sprectra_pchip_baseline_corrected_BC.csv',sep=',', index_col=0)
df_spectra_pw = pd.read_csv('../Dissertation_projects/Data/CDOM/sprectra_pchip_baseline_corrected_PW.csv',sep=',', index_col=0)
df_spectra_wc = pd.read_csv('../Dissertation_projects/Data/CDOM/sprectra_pchip_baseline_corrected_WC.csv',sep=',', index_col=0)
df_spectra_rz = pd.read_csv('../Dissertation_projects/Data/CDOM/sprectra_pchip_baseline_corrected_RZ.csv',sep=',', index_col=0)

df_spectra_bc.columns = ['a' + col for col in df_spectra_bc.columns]
df_spectra_pw.columns = ['a' + col for col in df_spectra_pw.columns]
df_spectra_wc.columns = ['a' + col for col in df_spectra_wc.columns]
df_spectra_rz.columns = ['a' + col for col in df_spectra_rz.columns]

In [9]:
#get BC data sets 
df_cdom_bc = df_cdom[df_cdom['Type'] == 'LANDSYR']
df_iron_bc = df_iron[df_iron['Type'] == 'BC']
df_doc_bc = df_doc[df_doc['Type'] == 'BC']

#get PW data sets 
df_cdom_pw = df_cdom[df_cdom['Type'] == 'PW']
df_iron_pw = df_iron[df_iron['Type'] == 'PW']
df_doc_pw = df_doc[df_doc['Type'] == 'PW']

#get WC data sets 
df_cdom_wc = df_cdom[df_cdom['Type'] == 'NISK']
df_iron_wc = df_iron[df_iron['Type'] == 'WC']
df_doc_wc = df_doc[df_doc['Type'] == 'WC']

#get RHZ data 
df_cdom_rz = df_cdom[df_cdom['Type'] == 'RHZ']
df_iron_rz = df_iron[df_iron['Type'].isin(['RZ', 'GBRZ'])]
df_doc_rz = df_doc[df_doc['Type'] == 'RZ']

In [10]:
#Put all the BC data together 
#remove repeat rows 
df_cdom_bc_cl = df_cdom_bc.drop(['Date', 'Type', 'Notes', 'date_time', 'code', 
                                 'Trip', 'Station', 'Lat', 'Lon', 'Water_depth', 'Sample_depth',
                                 'Sample_depth_units', 'Wavguide_Path_lengths', 'BC_time',
                                 'BC_Chamber_vol_(L)', 'Waveguide_used', 'Flagged','file_id'], axis=1)
df_iron_bc_cl = df_iron_bc.drop(['Treatment', 'Campaign', 'Location', 'Type', 'code'], axis=1)

#combind DOC and CDOM data 
df_bc = df_spectra_bc.join(df_doc_bc)
df_bc = df_bc.join(df_cdom_bc_cl)

#match up the inxdes for the merge 
df_iron_bc_cl['sample_index'] = df_iron_bc_cl.index.astype(str)
df_bc.index = df_bc.index.astype(str)

#combind the iron data 
df_bc = df_bc.join(df_iron_bc_cl)

#Put all the PW data together 
#remove repeat rows 
df_cdom_pw_cl = df_cdom_pw.drop(['Date', 'Type', 'Notes', 'date_time', 'code', 
                                 'Trip', 'Station', 'Lat', 'Lon', 'Water_depth', 'Sample_depth',
                                 'Sample_depth_units', 'Wavguide_Path_lengths', 'BC_time',
                                 'BC_Chamber_vol_(L)', 'Waveguide_used', 'Flagged', 'file_id'], axis=1)
df_iron_pw_cl = df_iron_pw.drop(['Treatment', 'Campaign', 'Location', 'Type', 'code'], axis=1)

#combind DOC and CDOM data 
df_pw = df_spectra_pw.join(df_doc_pw)
df_pw = df_pw.join(df_cdom_pw_cl)

#match up the inxdes for the merge 
df_iron_pw_cl['sample_index'] = df_iron_pw_cl.index.astype(str)
df_pw.index = df_pw.index.astype(str)

#combind the iron data 
df_pw = df_pw.join(df_iron_pw_cl)

#Put all the WC data together 
#remove repeat rows 
df_cdom_wc_cl = df_cdom_wc.drop(['Date', 'Type', 'Notes', 'date_time', 'code', 
                                 'Trip', 'Station', 'Lat', 'Lon', 'Water_depth', 'Sample_depth',
                                 'Sample_depth_units', 'Wavguide_Path_lengths', 'BC_time',
                                 'BC_Chamber_vol_(L)', 'Waveguide_used', 'Flagged', 'file_id'], axis=1)
df_iron_wc_cl = df_iron_wc.drop(['Treatment', 'Campaign', 'Location', 'Type', 'code'], axis=1)

#combind DOC and CDOM data 
df_wc = df_spectra_wc.join(df_doc_wc)
df_wc = df_wc.join(df_cdom_wc_cl)

#match up the inxdes for the merge 
df_iron_wc_cl['sample_index'] = df_iron_wc_cl.index.astype(str)
df_wc.index = df_wc.index.astype(str)

#combind the iron data 
df_wc = df_wc.join(df_iron_wc_cl)

#Put all the RZ data together 
#remove repeat rows 
df_cdom_rz_cl = df_cdom_rz.drop(['Date', 'Type', 'Notes', 'date_time', 'code', 
                                 'Trip', 'Station', 'Lat', 'Lon', 'Water_depth', 'Sample_depth',
                                 'Sample_depth_units', 'Wavguide_Path_lengths', 'BC_time',
                                 'BC_Chamber_vol_(L)', 'Waveguide_used', 'Flagged', 'file_id'], axis=1)
df_iron_rz_cl = df_iron_rz.drop(['Treatment', 'Campaign', 'Location', 'Type', 'code'], axis=1)

#combind DOC and CDOM data 
df_rz = df_spectra_rz.join(df_doc_rz)
df_rz = df_rz.join(df_cdom_rz_cl)

#match up the inxdes for the merge 
df_iron_rz_cl['sample_index'] = df_iron_rz_cl.index.astype(str)
df_rz.index = df_rz.index.astype(str)

#combind the iron data 
df_rz = df_rz.join(df_iron_rz_cl)

df_wc

,a250,a251,a252,a253,a254,a255,a256,a257,a258,a259,...,FeT_Depth,FeT_Time,Fe2_Date,Fe2_uM,Fe2_mgL,Fe2_Notes,Fe2_Depth,Fe2_Time,Fe3_uM,sample_index
Sample_ID,,,,,,,,,,,,,,,,,,,,,
212301,1.966948,1.907249,1.833414,1.793763,1.755289,1.716594,1.684068,1.628457,1.590463,1.587945,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212301
212302,3.208782,3.040563,2.953443,2.847333,2.724667,2.605936,2.507259,2.378898,2.292105,2.175383,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212303,2.947587,2.900612,2.827761,2.722262,2.646043,2.570617,2.448917,2.374863,2.299453,2.211905,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212303
212304,2.226661,2.195277,2.149554,2.083045,2.009987,1.961411,1.903917,1.842733,1.811708,1.740604,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212304
212305,2.966302,2.873939,2.805026,2.703202,2.626770,2.531954,2.479274,2.372418,2.304180,2.236926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OWENS,8.988554,8.458015,8.510507,8.659872,8.568503,8.757216,8.699847,8.697351,8.793161,8.761617,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OWENS,8.376166,8.496002,8.388993,8.659813,8.609495,8.745414,8.651719,8.651151,8.847863,8.800084,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OWENS,8.629897,8.474281,8.486884,8.579138,8.514927,8.827594,8.576258,8.658776,8.771611,8.834906,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
stations = ['St.5B', 'St.4', 'St.MK', 'St.7', 'St.16', 'St.2', 'St.9', 'St.14', 'St.15', 'St.13', 'St.11', 'St.12']
trips = ['GOM2021', 'GOM21FA', 'GOM22SP', 'GOM22SU']

# Setup colormap and CDOM Spectra
fig, axes = plt.subplots(len(trips), len(stations), figsize=(len(stations)*10, len(trips)*20))

# Determine the maximum depth for consistent plotting

txt_s = 23

for i, trip in enumerate(trips):
    for j, station in enumerate(stations):
        ax = axes[i, j]
        subset_df = df_wc[(df_wc['Campaign'] == trip) & (df_wc['Location'] == station)]

        if not subset_df.empty:
            # Variables
            doc_uM = np.asarray(subset_df['NPOC_uM'], dtype=float).ravel()
            tn_uM = np.asarray(subset_df['TN_uM'], dtype=float).ravel()
            doc_depth = np.asarray(subset_df['Depth'], dtype=float).ravel()
            fe2_uM = np.asarray(subset_df['Fe2_uM'], dtype=float).ravel()
            fe3_uM = np.asarray(subset_df['Fe3_uM'], dtype=float).ravel()
            fe_depth = np.asarray(subset_df['Fe2_Depth'], dtype=float).ravel()
            a350 = np.asarray(subset_df['a350'], dtype=float).ravel()
            a412 = np.asarray(subset_df['a412'], dtype=float).ravel()
            a550 = np.asarray(subset_df['a550'], dtype=float).ravel()
            spectra_depth = np.asarray(subset_df['Depth'], dtype=float).ravel()
            
            max_depth = max(subset_df['Depth'].max(), subset_df['Fe2_Depth'].max(), subset_df['Depth'].max())
            
            # PLOT DOC
            ax.plot(doc_uM, doc_depth, 'o', ls='-', c='black')
            ax.set_xlim(0, df_wc['NPOC_uM'].max())
            ax.set_ylim(max_depth, -1)  # Adjust for consistent depth
            ax.axhline(y=0, color='grey', linestyle='-', label='Sediment Water Interface')
            ax.set_xlabel('DOC (μM)', size=txt_s, c='black')
            if j == 0:
                ax.set_ylabel('Depth (cm)', size=txt_s)

            # Plot CDOM
            ax2 = ax.twiny()
            ax2.plot(a350, spectra_depth, 'o', ls='-', c='purple')
            ax2.set_xlim(0, df_wc['a350'].max()/4)
            ax2.xaxis.set_ticks_position('bottom')
            ax2.xaxis.set_label_position('bottom')
            ax2.spines['bottom'].set_position(('outward', 50))
            ax2.set_xlabel(r'$a_{\mathrm{CDOM}}$ 350 ' '\n' '(nm) (m$^{-1}$)', size=txt_s, c='purple')
            
            # Plot CDOM
            ax3 = ax.twiny()
            ax3.plot(a412, spectra_depth, 'o', ls='-', c='g')
            ax3.set_xlim(0, df_wc['a412'].max()/4)
            ax3.xaxis.set_ticks_position('bottom')
            ax3.xaxis.set_label_position('bottom')
            ax3.spines['bottom'].set_position(('outward', 100))
            ax3.set_xlabel(r'$a_{\mathrm{CDOM}}$ 412 ' '\n' '(nm) (m$^{-1}$)', size=txt_s, c='g')
            
            # Plot CDOM
            ax4 = ax.twiny()
            ax4.plot(a550, spectra_depth, 'o', ls='-', c='y')
            ax4.set_xlim(0, df_wc['a550'].max()/4)
            ax4.xaxis.set_ticks_position('bottom')
            ax4.xaxis.set_label_position('bottom')
            ax4.spines['bottom'].set_position(('outward', 150))
            ax4.set_xlabel(r'$a_{\mathrm{CDOM}}$ 550 ' '\n' '(nm) (m$^{-1}$)', size=txt_s, c='y')

            # Plot iron 2
            ax5 = ax.twiny()
            ax5.plot(fe2_uM, fe_depth, 'o', ls='-', c='b')
            ax5.set_xlim(0, df_wc['Fe2_uM'].max())
            ax5.xaxis.set_ticks_position('bottom')
            ax5.xaxis.set_label_position('bottom')
            ax5.spines['bottom'].set_position(('outward', 200))
            ax5.set_xlabel('Fe(II) (μM)', size=txt_s, c='b')

            # Plot iron 3
            ax6 = ax.twiny()
            ax6.plot(fe3_uM, fe_depth, 'o', ls='-', c='r')
            ax6.set_xlim(0, df_wc['Fe3_uM'].max())
            ax6.xaxis.set_ticks_position('bottom')
            ax6.xaxis.set_label_position('bottom')
            ax6.spines['bottom'].set_position(('outward', 230))
            ax6.set_xlabel('Fe(III) (μM)', size=txt_s, c='r')

            # Additional plots and configurations...


            # Turn off other axis
            if j != 0:
                ax.set_yticklabels([])
                ax2.set_yticklabels([])
                ax3.set_yticklabels([])
                ax4.set_yticklabels([])
                ax5.set_yticklabels([])
                ax6.set_yticklabels([])
                # ... Add other axes ...

            # Meta info for title
            water_depth = df_meta[(df_meta['Trip'] == trip) & (df_meta['Station'] == station)]
            water_depth = water_depth['Water_depth'].mean()
            water_depth = round(water_depth, 1)
            ax.set_title(f'{trip} - {station}: {water_depth} meters deep', size=txt_s*1.5)
        else:
            ax.axis('off')

        ax.set_xlabel('Depth (M)', size=txt_s, c='black')
        ax.xaxis.set_tick_params(labelsize=txt_s*.75)

fig.set_facecolor('w')
plt.tight_layout()
plt.show()


In [12]:
import matplotlib.pyplot as plt
import numpy as np

stations = ['St.5B', 'St.4', 'St.MK', 'St.7', 'St.16', 'St.2', 'St.9', 'St.14', 'St.15', 'St.13', 'St.11', 'St.12']
trips = ['GOM2021', 'GOM21FA', 'GOM22SP', 'GOM22SU']

# Configuration
txt_s = 30  # Size for most texts
title_size = 35  # For titles
point_size = 10  # Data point size

fig, axes = plt.subplots(len(trips), len(stations), figsize=(len(stations)*14, len(trips)*28))  # increased figsize for more room

for i, trip in enumerate(trips):
    for j, station in enumerate(stations):
        ax = axes[i, j]
        subset_df = df_wc[(df_wc['Campaign'] == trip) & (df_wc['Location'] == station)]

        if not subset_df.empty:
            # Extracted data
            max_depth = max(subset_df['Depth'].max(), subset_df['Fe2_Depth'].max(), subset_df['Depth'].max())

            # Variables
            doc_uM = np.asarray(subset_df['NPOC_uM'], dtype=float)
            doc_depth = np.asarray(subset_df['Depth'], dtype=float)
            fe2_uM = np.asarray(subset_df['Fe2_uM'], dtype=float)
            fe3_uM = np.asarray(subset_df['Fe3_uM'], dtype=float)
            fe_depth = np.asarray(subset_df['Fe2_Depth'], dtype=float)
            a350 = np.asarray(subset_df['a350'], dtype=float)
            a412 = np.asarray(subset_df['a412'], dtype=float)
            a550 = np.asarray(subset_df['a550'], dtype=float)
            spectra_depth = np.asarray(subset_df['Depth'], dtype=float)

            # DOC Plotting
            ax.plot(doc_uM, doc_depth, linestyle='-', c='black', markersize=point_size)
            ax.set_xlim(0, df_wc['NPOC_uM'].max())
            ax.set_ylim(max_depth, -1)
            ax.axhline(y=0, color='grey', linestyle='-', label='Sediment Water Interface')
            ax.set_xlabel('DOC (μM)', size=txt_s, c='black')
            if j == 0:
                ax.set_ylabel('Depth (cm)', size=txt_s)

            # CDOM a350
            ax2 = ax.twiny()
            ax2.plot(a350, spectra_depth, linestyle='-', c='purple', markersize=point_size)
            ax2.set_xlim(0, df_wc['a350'].max()/4)
            ax2.xaxis.set_ticks_position('bottom')
            ax2.xaxis.set_label_position('bottom')
            ax2.spines['bottom'].set_position(('outward', 70))  # Increase room for label
            ax2.set_xlabel(r'$a_{\mathrm{CDOM}}$ 350 ' '\n' '(nm) (m$^{-1}$)', size=txt_s, c='purple')
            
            # CDOM a412
            ax3 = ax.twiny()
            ax3.plot(a412, spectra_depth, linestyle='-', c='g', markersize=point_size)
            ax3.set_xlim(0, df_wc['a412'].max()/4)
            ax3.xaxis.set_ticks_position('bottom')
            ax3.xaxis.set_label_position('bottom')
            ax3.spines['bottom'].set_position(('outward', 150))
            ax3.set_xlabel(r'$a_{\mathrm{CDOM}}$ 412 ' '\n' '(nm) (m$^{-1}$)', size=txt_s, c='g')
            
            # CDOM a550
            ax4 = ax.twiny()
            ax4.plot(a550, spectra_depth, linestyle='-', c='y', markersize=point_size)
            ax4.set_xlim(0, df_wc['a550'].max()/4)
            ax4.xaxis.set_ticks_position('bottom')
            ax4.xaxis.set_label_position('bottom')
            ax4.spines['bottom'].set_position(('outward', 230))
            ax4.set_xlabel(r'$a_{\mathrm{CDOM}}$ 550 ' '\n' '(nm) (m$^{-1}$)', size=txt_s, c='y')

            # Iron 2
            ax5 = ax.twiny()
            ax5.plot(fe2_uM, fe_depth, linestyle='-', c='b', markersize=point_size)
            ax5.set_xlim(0, df_wc['Fe2_uM'].max())
            ax5.xaxis.set_ticks_position('bottom')
            ax5.xaxis.set_label_position('bottom')
            ax5.spines['bottom'].set_position(('outward', 310))
            ax5.set_xlabel('Fe(II) (μM)', size=txt_s, c='b')

            # Iron 3
            ax6 = ax.twiny()
            ax6.plot(fe3_uM, fe_depth, linestyle='-', c='r', markersize=point_size)
            ax6.set_xlim(0, df_wc['Fe3_uM'].max())
            ax6.xaxis.set_ticks_position('bottom')
            ax6.xaxis.set_label_position('bottom')
            ax6.spines['bottom'].set_position(('outward', 390))
            ax6.set_xlabel('Fe(III) (μM)', size=txt_s, c='r')

            # Title
            water_depth = df_meta[(df_meta['Trip'] == trip) & (df_meta['Station'] == station)]
            water_depth = water_depth['Water_depth'].mean()
            water_depth = round(water_depth, 1)
            ax.set_title(f'{trip} - {station}: {water_depth} meters deep', size=title_size)
        else:
            ax.axis('off')

        # Adjust x-axis labels and ticks for bottom-most plots
        if i == len(trips) - 1:
            ax.set_xlabel('Depth (M)', size=txt_s, c='black')
            ax.xaxis.set_tick_params(labelsize=txt_s*0.75)

fig.set_facecolor('w')
plt.tight_layout()
plt.show()
